# Daily IceNet CLI Usage

## Context
### Purpose
The IceNet library provides the ability to download, process, train and predict from end to end via a set of command-line interfaces.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentraion forecasts.

### Highlights
The key features of an end to end run are: 
* Download 
* Process
* Train
* Predict

### Contributions
#### Notebook
James Byrne (author)

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Modelling funding
Funded by the Alan Turing Institute

## Install and load libraries

## Load data

## Modelling

## Outputs

## Summary

## Version
- Notebook:
- Codebase: